# Question A:

There are four heartbeat sounds in the data/audio folder (heartbeat1, heartbeat2, heartbeat3, and heartbeat4). Two sounds are recorded from the same person at the same condition, which you need to find out using time-series data visualization and a dynamic time warping algorithm.

(i) Visualize each sound clip and use visual perception to determine which two clips can be similar (write your thoughts and observation to receive full credit).

(ii) Use fastdtw library (implements DTW algorithm) and calculate the pairwise distances between two clips. In total, there are 6 pairs. Then, based on the distance scores, predict which two clips are similar.

## Import Essential Libraries

In [1]:
# python visualization libraries
import matplotlib.pyplot as plt
import matplotlib as mpl
# from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import plotly.express as px
%matplotlib inline

import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as subplots

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Install the [`fastdtw`](https://pypi.org/project/fastdtw/) PyPi library;

In [2]:
!pip install fastdtw

In [3]:
# test if import works
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

## Mount Google Drive

In [5]:
#@title
import os

# mount drive
from google.colab import drive
drive.mount('/content/drive')


# edit this path if needed
my_path = '/content/drive/My Drive/Colab Notebooks/'

# change to this path
os.chdir(my_path)

# verify present working directory. It should be identical to 'my_path'
!pwd

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks


## Loading/Playing Audio Clips

In [28]:
# Read stored audio files for comparison

fs1, data1 = wavfile.read("audio/heartbeat1.wav")
fs2, data2 = wavfile.read("audio/heartbeat2.wav")
fs3, data3 = wavfile.read("audio/heartbeat3.wav")
fs4, data4 = wavfile.read("audio/heartbeat4.wav")

# Take the max values along axis
data1 = np.amax(data1, axis=1)
data2 = np.amax(data2, axis=1)
data3 = np.amax(data3, axis=1)
data4 = np.amax(data4, axis=1)


# Visualize each sound clip

In [29]:
# import audio-play library
import IPython
from IPython.display import Audio

display("[v1]", Audio("audio/heartbeat1.wav"))
display("[v2]", Audio("audio/heartbeat2.wav"))
display("[v3]", Audio("audio/heartbeat3.wav"))
display("[v4]", Audio("audio/heartbeat4.wav"))

# import subplot library
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=2, 
    cols=2, 
    subplot_titles = (
        "v1", 
        "v2", 
        "v3", 
        "v4"
    )
)

# show each audio gram

fig.add_trace(
    go.Scatter(y = data1),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(y = data2),
    row=1, col=2,    
)


fig.add_trace(
    go.Scatter(y = data3),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(y = data4),
    row=2, col=2
)

fig.show()

Output hidden; open in https://colab.research.google.com to view.

From the above 4 visualizations, we can observe that clip v1 (trace 0) and clip v3 (trace 2) has a similar pattern.

# Comparing all the sound clips

### Comparing Clip 1 and Clip 2


In [20]:
## visualize
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

fig = go.Figure()

fig.add_trace(go.Scatter(y = data1 , opacity=0.5))
fig.add_trace(go.Scatter(y = data2, opacity=0.5))    

fig.show()
# Distance between clip 1 and clip 2
dist = fastdtw(data1, data2)
display('Distance between clip 1 and clip 2: ', dist[0])

Output hidden; open in https://colab.research.google.com to view.

### Comparing Clip 1 and Clip 3

In [18]:
fig = go.Figure()

fig.add_trace(go.Scatter(y = data1 , opacity=0.5))    
fig.add_trace(go.Scatter(y = data3, opacity=0.5))    

fig.show()

# Distance between clip 1 and clip 3
dist = fastdtw(data1, data3)
display('Distance between clip 1 and clip 3: ', dist[0])

Output hidden; open in https://colab.research.google.com to view.

### Comparing Clip 1 and Clip 4





In [21]:
fig = go.Figure()

fig.add_trace(go.Scatter(y = data1 , opacity=0.5))    
fig.add_trace(go.Scatter(y = data4, opacity=0.5))    

fig.show()

# Distance between clip 1 and clip 4
dist = fastdtw(data1, data4)
display('Distance between clip 1 and clip 4: ', dist[0])

Output hidden; open in https://colab.research.google.com to view.

### Comparing Clip 2 and Clip 3


In [23]:
fig = go.Figure()

fig.add_trace(go.Scatter(y = data1 , opacity=0.5))    
fig.add_trace(go.Scatter(y = data4, opacity=0.5))    

fig.show()

# Distance between clip 2 and clip 3
dist = fastdtw(data2, data3)
display('Distance between clip 2 and clip 3: ', dist[0])

Output hidden; open in https://colab.research.google.com to view.

### Comparing Clip 3 and Clip 4

In [26]:
fig = go.Figure()

fig.add_trace(go.Scatter(y = data1 , opacity=0.5))    
fig.add_trace(go.Scatter(y = data4, opacity=0.5))    

fig.show()

# Distance between clip 3 and clip 4
dist = fastdtw(data3, data4)
display('Distance between clip 3 and clip 4: ', dist[0])

Output hidden; open in https://colab.research.google.com to view.

### Comparing Clip 2 and Clip 4

In [27]:
fig = go.Figure()

fig.add_trace(go.Scatter(y = data1 , opacity=0.5))    
fig.add_trace(go.Scatter(y = data4, opacity=0.5))    

fig.show()

# Distance between clip 2 and clip 4
dist = fastdtw(data2, data4)
display('Distance between clip 2 and clip 4: ', dist[0])

Output hidden; open in https://colab.research.google.com to view.

### Observation
By using `fastdtw`, we can quickly calculate the distance between two different time series - in this case audio patterns.  

| Base | Query | Distance |
| ---- | ----- | -------- |
| Clip 1 | Clip 2 |  667014528.0 |
|        | Clip 3 |  227167723.0 |
|        | Clip 4 | 1604991493.0 |
| Clip 2 | Clip 3 |  496809647.0 |
|        | Clip 4 | 1802474413.0 |
| Clip 3 | Clip 4 | 1496812932.0 |

Observations:
* Clips 1 and 3 have the shortest distance as the audio clips have intonations

* Clips 2 and 4 have the longest distance due to the extremely exagerated intonation and speed.